In [2]:
import rasterio

#url = "https://landsat-pds.s3.amazonaws.com/c1/L8/226/077/LC08_L1GT_226077_20191005_20191005_01_RT/LC08_L1GT_226077_20191005_20191005_01_RT_B3.TIF"
archivo = "datos/LC08_L1GT_226077_20191005_20191005_01_RT_B3.tiff"

banda3 = rasterio.open(archivo)

In [76]:
print(banda3.name)
print(banda3.crs)
print(banda3.crs.wkt)
print(banda3.indexes)
print(banda3.shape)

datos/LC08_L1GT_226077_20191005_20191005_01_RT_B3.tiff
EPSG:32621
PROJCS["WGS 84 / UTM zone 21N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-57],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32621"]]
(1,)
(7741, 7651)


In [5]:
print(banda3.profile)

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 7651, 'height': 7741, 'count': 1, 'crs': CRS.from_dict(init='epsg:32621'), 'transform': Affine(30.0, 0.0, 317385.0,
       0.0, -30.0, -2599785.0), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'deflate', 'interleave': 'band'}


In [ ]:
%matplotlib inline
from rasterio.plot import show


show(banda3)

In [ ]:
show(banda3.read(1), transform=banda3.transform, cmap='Blues')

In [13]:
banda3.read(1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [ ]:

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,10))

img = ax.imshow(banda3.read(1), cmap="viridis")
fig.colorbar(img, ax=ax)

In [ ]:
date = '2019-10-05'
url = 'http://landsat-pds.s3.amazonaws.com/c1/L8/226/077/LC08_L1GT_226077_20191005_20191005_01_RT/'
redband = 'LC08_L1GT_226077_20191005_20191005_01_RT_B{}.TIF'.format(4)
nirband = 'LC08_L1GT_226077_20191005_20191005_01_RT_B{}.TIF'.format(5)

figure = plt.figure(figsize=(10,10))

with rasterio.open(url+redband) as src:
    profile = src.profile
    oviews = src.overviews(1) # list of overviews from biggest to smallest
    oview = oviews[1]  # Use second-highest resolution overview
    print('Decimation factor= {}'.format(oview))
    red = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

plt.imshow(red)
plt.colorbar()
plt.title('{}\nRed {}'.format(redband, red.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

In [ ]:
with rasterio.open(url+nirband) as src:
    oviews = src.overviews(1) # list of overviews from biggest to smallest
    oview = oviews[1]  # Use second-highest resolution overview
    nir = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

plt.imshow(nir)
plt.colorbar()
plt.title('{}\nNIR {}'.format(nirband, nir.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

In [ ]:
def calc_ndvi(nir,red):
    '''Calculate NDVI from integer arrays'''
    nir = nir.astype('f4')
    red = red.astype('f4')
    ndvi = (nir - red) / (nir + red)
    return ndvi

figure = plt.figure(figsize=(10,10))

ndvi = calc_ndvi(nir,red)
plt.imshow(ndvi, cmap='RdYlGn')
plt.colorbar()
plt.title('NDVI {}'.format(date))
plt.xlabel('Column #')
plt.ylabel('Row #')